In [1]:
%%bash
# 不要かも

git clone https://github.com/drjerry/gutenberg-index.git
cd gutenberg-index
pip install --user lxml
./index.sh

cache/epub/0/pg0.rdf
cache/epub/10000/pg10000.rdf
cache/epub/10001/pg10001.rdf
cache/epub/10002/pg10002.rdf
cache/epub/10003/pg10003.rdf
cache/epub/10004/pg10004.rdf
cache/epub/10005/pg10005.rdf
cache/epub/10006/pg10006.rdf
cache/epub/10007/pg10007.rdf
cache/epub/10008/pg10008.rdf
cache/epub/10009/pg10009.rdf
cache/epub/1000/pg1000.rdf
cache/epub/10010/pg10010.rdf
cache/epub/10011/pg10011.rdf
cache/epub/10012/pg10012.rdf
cache/epub/10013/pg10013.rdf
cache/epub/10014/pg10014.rdf
cache/epub/10015/pg10015.rdf
cache/epub/10016/pg10016.rdf
cache/epub/10017/pg10017.rdf
cache/epub/10018/pg10018.rdf
cache/epub/10019/pg10019.rdf
cache/epub/1001/pg1001.rdf
cache/epub/10020/pg10020.rdf
cache/epub/10021/pg10021.rdf
cache/epub/10022/pg10022.rdf
cache/epub/10023/pg10023.rdf
cache/epub/10024/pg10024.rdf
cache/epub/10025/pg10025.rdf
cache/epub/10026/pg10026.rdf
cache/epub/10027/pg10027.rdf
cache/epub/10028/pg10028.rdf
cache/epub/10029/pg10029.rdf
cache/epub/1002/pg1002.rdf
cache/epub/10030/pg10030.rdf

Cloning into 'gutenberg-index'...
--2019-03-03 23:36:35--  http://www.gutenberg.org/cache/epub/feeds/rdf-files.tar.bz2
www.gutenberg.org (www.gutenberg.org) をDNSに問いあわせています... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 45680778 (44M) [application/x-bzip2]
`/home/mitsuhisa.ohta/gutenberg/rdf-files.tar.bz2' に保存中

     0K .......... .......... .......... .......... ..........  0%  135K 5m30s
    50K .......... .......... .......... .......... ..........  0%  237K 4m18s
   100K .......... .......... .......... .......... ..........  0%  300K 3m42s
   150K .......... .......... .......... .......... ..........  0% 1.78M 2m52s
   200K .......... .......... .......... .......... ..........  0%  291K 2m48s
   250K .......... .......... .......... .......... ..........  0% 4.29M 2m21s
   300K .......... .......... .......... .......... ..........  0% 5.47M 2m2s
   350K ....

In [2]:
import requests
from bs4 import BeautifulSoup
from collections import Counter, defaultdict
from gutenberg.acquire.text import UnknownDownloadUriException
import re
from gensim.utils import tokenize
import random
import nltk
from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers
import os
import glob
import json

In [3]:
with open('data/gutenberg_index.json') as fin:
    authors = json.load(fin)
recent = [x for x in authors if 'birthdate' in x and x['birthdate'] > 1830]
[(x['name'], x['birthdate'], x['english_books']) for x in recent[:5]]

FileNotFoundError: [Errno 2] No such file or directory: 'data/gutenberg_index.json'

In [ ]:
from gutenberg.query import list_supported_metadatas
print(list_supported_metadatas())

In [ ]:
PARAGRAPH_SPLIT_RE = re.compile(r'\n *\n+')

def extract_conversations(text, quote='"'):
    paragraphs = PARAGRAPH_SPLIT_RE.split(text.strip())
    conversations = [['']]
    for paragraph in paragraphs:
        chunks = paragraph.replace('\n', ' ').split(quote)
        for i in range((len(chunks) + 1) // 2):
            if (len(chunks[i * 2]) > 100 or len(chunks) == 1) and conversations[-1] != ['']:
                if conversations[-1][-1] == '':
                    del conversations[-1][-1]
                conversations.append([''])
            if i * 2 + 1 < len(chunks):
                chunk = chunks[i * 2 + 1]
                if chunk:
                    if conversations[-1][-1]:
                        if chunk[0] >= 'A' and chunk[0] <= 'Z':
                            if conversations[-1][-1].endswith(','):
                                conversations[-1][-1] = conversations[-1][-1][:-1]
                            conversations[-1][-1] += '.'
                        conversations[-1][-1] += ' '
                    conversations[-1][-1] += chunk
        if conversations[-1][-1]:
            conversations[-1].append('')

    return [x for x in conversations if len(x) > 1]


conversations = extract_conversations(strip_headers(load_etext(10008).strip()))
sum(len(x) for x in conversations)

In [ ]:
LATIN_1_CHARS = (
    (u'\xe2\x80\x99', "'"),
    (u'\xc3\xa9', 'e'),
    (u'\xe2\x80\x90', '-'),
    (u'\xe2\x80\x91', '-'),
    (u'\xe2\x80\x92', '-'),
    (u'\xe2\x80\x93', '-'),
    (u'\xe2\x80\x94', '-'),
    (u'\xe2\x80\x94', '-'),
    (u'\xe2\x80\x98', "'"),
    (u'\xe2\x80\x9b', "'"),
    (u'\xe2\x80\x9c', '"'),
    (u'\xe2\x80\x9c', '"'),
    (u'\xe2\x80\x9d', '"'),
    (u'\xe2\x80\x9e', '"'),
    (u'\xe2\x80\x9f', '"'),
    (u'\xe2\x80\xa6', '...'),
    (u'\xe2\x80\xb2', "'"),
    (u'\xe2\x80\xb3', "'"),
    (u'\xe2\x80\xb4', "'"),
    (u'\xe2\x80\xb5', "'"),
    (u'\xe2\x80\xb6', "'"),
    (u'\xe2\x80\xb7', "'"),
    (u'\xe2\x81\xba', "+"),
    (u'\xe2\x81\xbb', "-"),
    (u'\xe2\x81\xbc', "="),
    (u'\xe2\x81\xbd', "("),
    (u'\xe2\x81\xbe', ")")
)

books = 0
for author in recent[:1000]:
    for book in author['books']:
        books += 1
        try:
            txt = strip_headers(load_etext(int(book[0]))).strip()
        except UnknownDownloadUriException:
            continue
        for ch1, ch2 in LATIN_1_CHARS:
            txt = txt.replace(ch1, ch2)
        conversations += extract_conversations(txt)

print(len(conversations), books)

In [ ]:
with open('gutenberg.txt', 'w') as fout:
    for conv in conversations:
        fout.write('\n'.join(conv) + '\n\n')

In [ ]:
RE_TOKEN = re.compile('(\w+|\?)', re.UNICODE)
token_counter = Counter()
with open('gutenberg.txt') as fin:
    for line in fin:
        line = line.lower().replace('_', ' ')
        token_counter.update(RE_TOKEN.findall(line))
with open('gutenberg.tok', 'w') as fout:
    for token, count in token_counter.items():
        fout.write('%s\t%d\n' % (token, count))

In [ ]:
token_counter['?']

In [ ]:
PAT_ALPHABETIC.findall(conv[0])

In [ ]:
RE